### 📌 Passo 1: Criar as Dimensões

In [2]:
import pandas as pd
import os
import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv

# 🔹 Carregar variáveis de ambiente do .env
load_dotenv()

# 🔹 Configurar conexão com o PostgreSQL (Staging Area)
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

DATABASE_URL_POSTGRES = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}?sslmode=require"

# 🔹 Testar conexão com PostgreSQL antes de criar engine
try:
    conn_test = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        sslmode="require"
    )
    conn_test.close()
    print("✅ Conexão com PostgreSQL testada com sucesso!")
    engine_pg = create_engine(DATABASE_URL_POSTGRES, pool_size=5, max_overflow=10)
    conn_pg = engine_pg.connect()
except Exception as e:
    print(f"❌ Erro ao conectar ao PostgreSQL: {e}")
    conn_pg = None  # Evita erro de variável indefinida

# 🔹 Configurar conexão com o MySQL (Destino das Dimensões)
DB_USER_MYSQL = os.getenv("DB_USER_MYSQL")
DB_PASSWORD_MYSQL = os.getenv("DB_PASSWORD_MYSQL")
DB_HOST_MYSQL = os.getenv("DB_HOST_MYSQL")
DB_PORT_MYSQL = os.getenv("DB_PORT_MYSQL")
DB_NAME_MYSQL = os.getenv("DB_NAME_MYSQL")

DATABASE_URL_MYSQL = f"mysql+pymysql://{DB_USER_MYSQL}:{DB_PASSWORD_MYSQL}@{DB_HOST_MYSQL}:{DB_PORT_MYSQL}/{DB_NAME_MYSQL}"

try:
    engine_mysql = create_engine(DATABASE_URL_MYSQL, pool_size=5, max_overflow=10)
    conn_mysql = engine_mysql.connect()
    print("✅ Conexão com MySQL estabelecida com sucesso!")
except Exception as e:
    print(f"❌ Erro ao conectar ao MySQL: {e}")
    exit()

# 🔹 Carregar dados da Staging Area
if conn_pg:
    try:
        df_stg = pd.read_sql("SELECT * FROM stg_acidentes", con=engine_pg)
        print("✅ Dados carregados da Staging Area com sucesso!")
    except Exception as e:
        print(f"❌ Erro ao carregar dados do PostgreSQL: {e}")
        df_stg = pd.DataFrame()  # Evita erro caso o dataframe não seja carregado
else:
    df_stg = pd.DataFrame()
    print("⚠️ Pulando carregamento de dados do PostgreSQL, pois a conexão falhou.")

# 🔹 Criar função para salvar dimensões no MySQL
def save_dimension(df, table_name, engine):
    """Salva uma dimensão no banco de destino e adiciona surrogate key (sk)"""
    if df.empty:
        print(f"⚠️ Dimensão {table_name} não foi carregada porque os dados estão vazios.")
        return
    try:
        df["sk"] = range(1, len(df) + 1)  # Criar Surrogate Key
        df.to_sql(table_name, con=engine, if_exists="replace", index=False)
        print(f"✅ Dimensão {table_name} carregada com sucesso no MySQL.")
    except Exception as e:
        print(f"❌ Erro ao salvar {table_name} no MySQL: {e}")

# 🔹 Criar Dimensões e salvar no MySQL
dim_situacao_acidente = pd.DataFrame({
    "tipo": ["ilesos", "levemente_feridos", "moderadamente_feridos", "gravemente_feridos", "mortos"]
})
save_dimension(dim_situacao_acidente, "dim_situacao_acidente", engine_mysql)

dim_tipo_veiculo = pd.DataFrame({
    "tipo_veiculo": ["automovel", "bicicleta", "caminhao", "moto", "onibus",
                     "outros", "tracao_animal", "transporte_de_cargas_especiais",
                     "trator_maquinas", "utilitarios"]
})
save_dimension(dim_tipo_veiculo, "dim_tipo_veiculo", engine_mysql)

if not df_stg.empty:
    dim_tipo_ocorrencia = df_stg[["tipo_de_ocorrencia"]].drop_duplicates().dropna().reset_index(drop=True)
    save_dimension(dim_tipo_ocorrencia, "dim_tipo_ocorrencia", engine_mysql)

    dim_sentido = df_stg[["sentido"]].drop_duplicates().dropna().reset_index(drop=True)
    save_dimension(dim_sentido, "dim_sentido", engine_mysql)

    dim_rodovia = df_stg[["concessionaria"]].drop_duplicates().dropna().reset_index(drop=True)
    save_dimension(dim_rodovia, "dim_rodovia", engine_mysql)

    dim_trecho = df_stg[["trecho"]].drop_duplicates().dropna().reset_index(drop=True)
    save_dimension(dim_trecho, "dim_trecho", engine_mysql)

# Fechar conexões, se existirem
if conn_pg:
    conn_pg.close()
conn_mysql.close()
print("✅ Conexões fechadas com sucesso!")


✅ Conexão com PostgreSQL testada com sucesso!
✅ Conexão com MySQL estabelecida com sucesso!
✅ Dados carregados da Staging Area com sucesso!
✅ Dimensão dim_situacao_acidente carregada com sucesso no MySQL.
✅ Dimensão dim_tipo_veiculo carregada com sucesso no MySQL.
✅ Dimensão dim_tipo_ocorrencia carregada com sucesso no MySQL.
✅ Dimensão dim_sentido carregada com sucesso no MySQL.
✅ Dimensão dim_rodovia carregada com sucesso no MySQL.
✅ Dimensão dim_trecho carregada com sucesso no MySQL.
✅ Conexões fechadas com sucesso!
